https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [5]:
import pandas as pd
import os
train_df = pd.read_csv("train_dataset.csv")
test_df = pd.read_csv("test_dataset.csv")

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [7]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="name",
    y_col="class",
    target_size=input_shape[:2],
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col="name",
    y_col="class",
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [8]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [13]:
len(train_df) // batch_size, len(train_generator)

(1562, 1563)

In [14]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1563/1563 [==============================] - 95s 61ms/step - loss: 1.7441 - accuracy: 0.3558 - val_loss: 1.3794 - val_accuracy: 0.4976
Epoch 2/10
1563/1563 [==============================] - 35s 23ms/step - loss: 1.4681 - accuracy: 0.4647 - val_loss: 1.3555 - val_accuracy: 0.5091
Epoch 3/10
1563/1563 [==============================] - 34s 22ms/step - loss: 1.3658 - accuracy: 0.5074 - val_loss: 1.1469 - val_accuracy: 0.5865
Epoch 4/10
1563/1563 [==============================] - 37s 24ms/step - loss: 1.3067 - accuracy: 0.5262 - val_loss: 1.0986 - val_accuracy: 0.6018
Epoch 5/10
1563/1563 [==============================] - 35s 23ms/step - loss: 1.2714 - accuracy: 0.5425 - val_loss: 1.0880 - val_accuracy: 0.6007
Epoch 6/10
1563/1563 [==============================] - 38s 24ms/step - loss: 1.2390 - accuracy: 0.5558 - val_loss: 1.2139 - val_accuracy: 0.5707
Epoch 7/10
1563/1563 [==========================